In [1]:
import os
os.chdir("..")

In [2]:
from skin_lesion_cad.training.models.swin import SwinModel

/home/user0/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
chkp_pretrained = "outputs_backup/swin_tiny_augm_remove_weird_color/lightning_logs/version_1/checkpoints/epoch=14-valid_acc=0.8949.ckpt"
model = SwinModel.load_from_checkpoint(chkp_pretrained).model

In [4]:
model

SwinTransformer(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): Permute()
      (2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (1): Sequential(
      (0): SwinTransformerBlockV2(
        (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttentionV2(
          (qkv): Linear(in_features=96, out_features=288, bias=True)
          (proj): Linear(in_features=96, out_features=96, bias=True)
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=3, bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=96, out_features=384, bias=True)
          (1): GELU(approximate='

In [5]:
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, Normalize, ToTensor

def to_tensor(img):
    transform_fn = Compose([Resize(249, 3), CenterCrop(224), ToTensor(), Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    return transform_fn(img)

img1 = Image.open("data/raw/chall1/train/nevus/nev00003.jpg")
img1 =  to_tensor(img1)


/home/user0/.local/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [6]:
model.features[1][0].attn.qkv

Linear(in_features=96, out_features=288, bias=True)

In [7]:
def my_forward_wrapper(attn_obj):
    def my_forward(x):
        print(x.shape)
        B, N, C = x.shape
        qkv = attn_obj.qkv(x).reshape(B, N, 3, attn_obj.num_heads, C // attn_obj.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv.unbind(0)   # make torchscript happy (cannot use tensor as tuple)

        attn = (q @ k.transpose(-2, -1)) * attn_obj.scale
        attn = attn.softmax(dim=-1)
        attn = attn_obj.attn_drop(attn)
        attn_obj.attn_map = attn
        attn_obj.cls_attn_map = attn[:, :, 0, 2:]

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = attn_obj.proj(x)
        x = attn_obj.proj_drop(x)
        return x
    return my_forward


In [8]:
model.features[1]

Sequential(
  (0): SwinTransformerBlockV2(
    (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    (attn): ShiftedWindowAttentionV2(
      (qkv): Linear(in_features=96, out_features=288, bias=True)
      (proj): Linear(in_features=96, out_features=96, bias=True)
      (cpb_mlp): Sequential(
        (0): Linear(in_features=2, out_features=512, bias=True)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=512, out_features=3, bias=False)
      )
    )
    (stochastic_depth): StochasticDepth(p=0.0, mode=row)
    (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    (mlp): MLP(
      (0): Linear(in_features=96, out_features=384, bias=True)
      (1): GELU(approximate='none')
      (2): Dropout(p=0.0, inplace=False)
      (3): Linear(in_features=384, out_features=96, bias=True)
      (4): Dropout(p=0.0, inplace=False)
    )
  )
  (1): SwinTransformerBlockV2(
    (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    (attn): ShiftedWindowAtte

In [9]:
attn_obj = model.features[1][0].attn

In [10]:
attn_obj.num_heads

3

In [11]:
# reshape(B, N, 3, attn_obj.num_heads, C // attn_obj.num_heads)
activation = {}


In [12]:

# model.blocks[-1].attn.forward =
# model.features[1][0].attn.forward(to_tensor(img1).unsqueeze(0))
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook
# model.features[1][0].attn.proj.register_forward_hook(get_activation('qkv'))
# model.features[1][0].attn.qkv.register_forward_hook(get_activation('x'))
# model.features[5][1].attn.qkv
model.features[5][0].attn.proj.register_forward_hook(get_activation('abc'))
# model.features[1][0].attn.cpb_mlp.register_forward_hook(get_activation('f_conv1'))

# output = model(to_tensor(img1).unsqueeze(0))


In [13]:
activation.keys()

dict_keys([])

In [14]:

y = model(img1.unsqueeze(0))


In [15]:
attn_obj

ShiftedWindowAttentionV2(
  (qkv): Linear(in_features=96, out_features=288, bias=True)
  (proj): Linear(in_features=96, out_features=96, bias=True)
  (cpb_mlp): Sequential(
    (0): Linear(in_features=2, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=3, bias=False)
  )
)